In [19]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [20]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/"

In [21]:
chase = pd.read_adls(path + "out_lexisnexis_adhoc_LN_11449.csv", dtype = str, encoding='iso-8859-1')
print(chase.shape)

(1405323, 31)


In [22]:
chase.columns

Index(['TRANSACTION_ID', 'CUST_FIRST_NM', 'CUST_MID_INIT_TX', 'CUST_LAST_NM',
       'GOVT_ISSU_ID_NB', 'LINE_1_AD', 'LINE_2_AD', 'CITY_NM', 'STATE_PROV_CD',
       'PST_AREA_7_CD', 'PST_AREA_CD', 'channel', 'bus_name', 'app_date',
       'DATE_OF_BIRTH', 'decision', 'bizidscore', 'BD_Score', 'BD_Score1',
       'BD_Score2', 'BD_Score3', 'bus_addr1', 'bus_addr2', 'bus_city',
       'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',
       'HOME_PHONE', 'bad'],
      dtype='object')

In [23]:
chase.TRANSACTION_ID.is_unique

False

In [24]:
chase["seq"] = pd.Series(range(1, 1 + len(chase)))
chase["account"] = chase["seq"].astype(str).str.pad(7, fillchar="0")

In [40]:
# chase_2 = pd.read_adls(path + "out_lexisnexis_adhoc_LN202303_11677.csv", dtype = str, encoding='iso-8859-1')
# print(chase_2.shape)

(1296800, 32)


In [25]:
chase.columns = [x.lower() for x in chase.columns]
# chase_2.columns = [x.lower() for x in chase_2.columns]

In [42]:
# [x for x in chase_2.columns if x not in chase.columns]

['acct']

In [43]:
# chase_2["date"] = chase_2.app_date.str.slice(6, 10) + chase_2.app_date.str.slice(0, 2)
# freq(chase_2["date"])

,Count,Pct,Cuml Count,Cuml Pct
date,,,,
202202,130464.0,0.100605,130464.0,0.100605
202203,160225.0,0.123554,290689.0,0.224159
202204,144805.0,0.111663,435494.0,0.335822
202205,141767.0,0.109321,577261.0,0.445143
202206,142213.0,0.109665,719474.0,0.554807
202207,144347.0,0.111310,863821.0,0.666117
202208,153530.0,0.118391,1017351.0,0.784509
202209,144345.0,0.111309,1161696.0,0.895817
202210,135104.0,0.104183,1296800.0,1.000000


In [26]:
chase["date"] = chase.app_date.str.slice(2, 9)
freq(chase.date)

,Count,Pct,Cuml Count,Cuml Pct
date,,,,
APR2022,141668.0,0.100808,141668.0,0.100808
AUG2022,150515.0,0.107103,292183.0,0.207912
FEB2022,127674.0,0.090850,419857.0,0.298762
JAN2022,133301.0,0.094854,553158.0,0.393616
JUL2022,141482.0,0.100676,694640.0,0.494292
JUN2022,139344.0,0.099154,833984.0,0.593446
MAR2022,156867.0,0.111623,990851.0,0.705070
MAY2022,138793.0,0.098762,1129644.0,0.803832
NOV2022,2712.0,0.001930,1132356.0,0.805762


In [27]:
chase = chase[chase["date"].isin(["JAN2022", "FEB2022", "MAR2022", "APR2022", "MAY2022", "JUN2022"])]
print(chase.shape)
# chase_2 = chase_2[chase_2["date"].isin(["202202", "202203", "202204", "202205", "202206"])]
# print(chase_2.shape)

(837647, 34)


In [28]:
# We decided to define a record as booked if acct was populated AND decision = STP, Pend, or Approve
chase["LN_booked"] = np.where(chase["decision"].isin(["STP", "Pend", "Approve"]), 1, 0)
freq(chase["LN_booked"])

,Count,Pct,Cuml Count,Cuml Pct
LN_booked,,,,
0,44272.0,0.052853,44272.0,0.052853
1,793375.0,0.947147,837647.0,1.000000


In [12]:
# chase_2["LN_booked"] = np.where(chase_2["decision"].isin(["STP", "Pend", "Approve"]), 1, 0)
# freq(chase_2["LN_booked"])

In [29]:
chase.bad = chase.bad.astype(int)
bivariate(chase.LN_booked, chase.bad)

tag             bad                                               
stats             N      PctN     Sum      Mean       WoE       IV
LN_booked                                                         
0           44272.0  0.052853     0.0  0.000000  0.000000 -0.00000
1          793375.0  0.947147  1119.0  0.001410  0.053929  0.00283
Missing         0.0  0.000000     0.0  0.000000  0.000000  0.00000
Total      837647.0  1.000000  1119.0  0.001336  0.000000  0.00283

In [14]:
# freq(chase_2.LN_booked, chase_2.bad, cross = False)

In [30]:
chase = chase[chase["LN_booked"] == 1]
# chase_2 = chase_2[chase_2["LN_booked"] == 1]

In [31]:
freq(chase.bad)

,Count,Pct,Cuml Count,Cuml Pct
bad,,,,
0,792256.0,0.99859,792256.0,0.99859
1,1119.0,0.00141,793375.0,1.00000


In [28]:
freq(chase_2.bad)

,Count,Pct,Cuml Count,Cuml Pct
bad,,,,
0,667957.0,0.993806,667957.0,0.993806
1,4163.0,0.006194,672120.0,1.000000


In [13]:
stacked = pd.concat([chase, chase_2], ignore_index=True)
print(stacked.shape)
print(stacked.transaction_id.is_unique)

(1557121, 33)
False


In [ ]:
print(chase.transaction_id.is_unique)
print(chase_2.transaction_id.is_unique)

False
False


In [29]:
chase.columns

Index(['transaction_id', 'cust_first_nm', 'cust_mid_init_tx', 'cust_last_nm',
       'govt_issu_id_nb', 'line_1_ad', 'line_2_ad', 'city_nm', 'state_prov_cd',
       'pst_area_7_cd', 'pst_area_cd', 'channel', 'bus_name', 'app_date',
       'date_of_birth', 'decision', 'bizidscore', 'bd_score', 'bd_score1',
       'bd_score2', 'bd_score3', 'bus_addr1', 'bus_addr2', 'bus_city',
       'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',
       'home_phone', 'bad', 'date', 'LN_booked'],
      dtype='object')

In [30]:
bus_list = ['bus_name', 'app_date','bus_addr1', 'bus_addr2', 'bus_city', 'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',]
dedup = chase.drop_duplicates(bus_list, ignore_index=True)
dedup["src"] = "11449"
print(dedup.shape)
dedup_2 = chase_2.drop_duplicates(bus_list, ignore_index=True)
dedup["src"] = "11677"
print(dedup_2.shape)

/tmp/ipykernel_25357/2731914532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup["src"] = "11449"


(642595, 34)
(539363, 34)


/tmp/ipykernel_25357/2731914532.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup["src"] = "11677"


In [31]:
stacked = pd.concat([dedup, dedup_2], ignore_index=True)
print(stacked.shape)

(1181958, 35)


In [34]:
stacked["count"] = 1
stacked = stacked.fillna("")
temp_roll = stacked.groupby(by = bus_list)['count'].sum().reset_index(name='count')
temp_roll["count"].describe()

count    1181958.0
mean           1.0
std            0.0
min            1.0
25%            1.0
50%            1.0
75%            1.0
max            1.0
Name: count, dtype: float64

In [35]:
stacked = pd.concat([chase, chase_2], ignore_index = True)
print(stacked.shape)

(1465495, 34)


In [33]:
chase.columns

Index(['transaction_id', 'cust_first_nm', 'cust_mid_init_tx', 'cust_last_nm',
       'govt_issu_id_nb', 'line_1_ad', 'line_2_ad', 'city_nm', 'state_prov_cd',
       'pst_area_7_cd', 'pst_area_cd', 'channel', 'bus_name', 'app_date',
       'date_of_birth', 'decision', 'bizidscore', 'bd_score', 'bd_score1',
       'bd_score2', 'bd_score3', 'bus_addr1', 'bus_addr2', 'bus_city',
       'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',
       'home_phone', 'bad', 'seq', 'account', 'date', 'LN_booked'],
      dtype='object')

In [38]:
chase["count"] = 1
chase = chase.fillna("")
temp_roll = chase.groupby(["app_date", 'bus_name', 'bus_addr1', 'bus_addr2', 'bus_city', 'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', "bad"])['count'].sum().reset_index(name='count')
print(temp_roll["count"].describe())
temp_roll["count_2"] =1
temp_roll_2 = temp_roll.groupby(["app_date", 'bus_name', 'bus_addr1', 'bus_addr2', 'bus_city', 'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone',])['count_2'].sum().reset_index(name='count_2')
temp_roll_2["count_2"].describe()

count    640969.000000
mean          1.237774
std           0.525744
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          11.000000
Name: count, dtype: float64


count    640958.000000
mean          1.000017
std           0.004143
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           2.000000
Name: count_2, dtype: float64

In [49]:
chase = chase.sort_values(by = ["app_date", 'bus_name', 'bus_addr1', 'bus_addr2', 'bus_city', 'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', "bad"])
print(chase.shape)
chase = chase.drop_duplicates(subset = ["app_date", 'bus_name', 'bus_addr1', 'bus_addr2', 'bus_city', 'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone'], keep = "last")
print(chase.shape)
freq(chase.bad)

(793375, 36)
(640958, 36)


,Count,Pct,Cuml Count,Cuml Pct
bad,,,,
0,639840.0,0.998256,639840.0,0.998256
1,1118.0,0.001744,640958.0,1.000000


In [50]:
# stacked.to_adls(path + "chase_stacked_final.parquet")
chase.to_adls(path + "chase_stacked_final.parquet", overwrite = True)